<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/CV-2025/blob/main/Assignment_01/UQ_CIFAR-10N_Ensembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Uncertainty Quantification with CIFAR-10N and Ensembling
By *First name* *Second name*.

*Month, Day, 2025.*

## Problem Statement

Re-annotated versions of the CIFAR-10 and CIFAR-100 data which contains real-world human annotation errors. We show how these noise patterns deviate from the classically assumed ones and what the new challenges are. The website of CIFAR-N is available at [cifar-10-100n
](https://github.com/UCSC-REAL/cifar-10-100n/tree/main) project.

# Preparation of simulation models

## Import and Install Libraries

In [1]:
!pip install pytorch-lightning clearml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.10.1
    Uninstalling PyJWT-2.10.1:
      Successfully uninstalled PyJWT-2.10.1


In [2]:
#Pytorch modules
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision.datasets import CIFAR10
from torchvision import datasets, transforms, models
#scipy
from scipy.stats import mode
#sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Lightning & logging
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
#Data observation
import os
import sys
import pickle
import requests
from pathlib import Path
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#Logging
from clearml import Task

## Set the Models

### Simulation Settings

Check the current directory

In [3]:
os.getcwd() #returns the current working directory

'/kaggle/working'

In [4]:
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/")
print(f'CHECKPOINT_PATH: {CHECKPOINT_PATH}')

os.makedirs(CHECKPOINT_PATH, exist_ok=True)

CHECKPOINT_PATH: saved_models/


Set the reproducibility options

In [5]:
# Function for setting the seed to implement parallel tests
SEEDS =  [42, 0, 17, 9, 3, 16, 2]
SEED = 42 # random seed by default
pl.seed_everything(SEED)

# Determine the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prioritizes speed but may reduce precision
torch.set_float32_matmul_precision('high')

# # Ensure that all operations are deterministic on GPU (if used) for reproducibility
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# torch.use_deterministic_algorithms(True)

# torch.manual_seed(SEED)
# np.random.seed(SEED)

### Logging

To configure ClearML in your Colab environment, follow these steps:

---

*Step 1: Create a ClearML Account*
1. Go to the [ClearML website](https://clear.ml/).
2. Sign up for a free account if you don’t already have one.
3. Once registered, log in to your ClearML account.

---

*Step 2: Get Your ClearML Credentials*
1. After logging in, navigate to the **Settings** page (click on your profile icon in the top-right corner and select **Settings**).
2. Under the **Workspace** section, find your **+ Create new credentials**.
3. Copy these credentials for a Jupiter notebook into the code cell below.

---

*Step 3: Accessing the ClearML Dashboard*
1. Go to your ClearML dashboard (https://app.clear.ml).
2. Navigate to the **Projects** section to see your experiments.
3. Click on the experiment (e.g., `Lab_1`) to view detailed metrics, logs, and artifacts.

---

In [6]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZDDS36VPFVGPCW1X8UB2C8WRB89GTR
%env CLEARML_API_SECRET_KEY=n5QIFvW2MzcufzB_Pvsm1P09z2miidofpzoXOp7mbFxC7ER5afZRBOKfPjZfWsw7EPU

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=ZDDS36VPFVGPCW1X8UB2C8WRB89GTR
env: CLEARML_API_SECRET_KEY=n5QIFvW2MzcufzB_Pvsm1P09z2miidofpzoXOp7mbFxC7ER5afZRBOKfPjZfWsw7EPU


### Dataset

Summary

In [7]:
DATASET = 'CIFAR10N' # dataset with the real-world noise
# Can be 'clean_label', 'worse_label', 'aggre_label', 'random_label1', 'random_label2', 'random_label3'
NOISE_TYPE = 'clean_label'
LS = 0.4

NS = {
    'train': 45000,
    'val': 5000,
    'test': 10000
} # for MNIST

SIZE = 32 #image size 32 is original size, but a ViT needs 224
NUM_CLASSES = 10
CLASS_NAMES = ['plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Collect parameters

In [8]:
#Model parameters
LOSS_FUN = 'B' # 'CE', 'N', 'B', etc.
ARCHITECTURE = 'CNN' # 'CNN', 'ResNet50', 'ViT', etc.

#Collect the parameters (hyperparams and others)
im_size = SIZE if ARCHITECTURE == 'CNN' else 224
hparams = {
    "seed": SEED,
    "lr": 0.001,
    'weight_decay': 0.0,
    "dropout": 0.0,
    "bs": 128,
    "num_workers": 0,
    "num_epochs": 20,
    "criterion": LOSS_FUN,
    "architecture": ARCHITECTURE,
    'freeze': True,
    "num_samples": NS,
    "im_size": im_size,
    "mean": np.array([0.4914, 0.4822, 0.4465]),
    "std": np.array([0.2470, 0.2435, 0.2616]),
    'randResCrop': {'size': (im_size, im_size), 'scale': (0.8, 1.0), 'ratio': (0.9, 1.1)},
    'label_smoothing': LS,
    "n_classes": NUM_CLASSES,
    "noise_path": './data/CIFAR-10_human.pt',
    "noise_type": NOISE_TYPE  # Can be 'clean_label', 'worse_label', 'aggre_label', etc.
}

#Visualization
vis_params = {
    'fig_size': 5,
    'num_samples': 5,
    'num_bins': 50,
}

## Functions

### Lightning

Data module

In [9]:
def download_file(url, save_path):
    """Download a file from a URL and save it to the specified path."""
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure directory exists
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"File downloaded and saved to {save_path}")
    else:
        raise Exception(f"Failed to download file from {url}. Status code: {response.status_code}")

In [10]:
class CIFAR10(datasets.CIFAR10):
    """CIFAR10 dataset with noisy labels."""
    def __init__(self, root, train=True, transform=None, target_transform=None,
                 download=False, noise_type=None, noise_path=None, is_human=True):
        super().__init__(root, train=train, transform=transform,
                         target_transform=target_transform, download=download)
        self.noise_type = noise_type
        self.noise_path = noise_path
        self.is_human = is_human

        if self.train and self.noise_type is not None:
            self.load_noisy_labels()

    def load_noisy_labels(self):
        noise_file = torch.load(self.noise_path)
        if isinstance(noise_file, dict):
            if "clean_label" in noise_file.keys():
                clean_label = torch.tensor(noise_file['clean_label'])
                assert torch.sum(torch.tensor(self.targets) - clean_label) == 0
                print(f'Loaded {self.noise_type} from {self.noise_path}.')
                print(f'The overall noise rate is {1 - np.mean(clean_label.numpy() == noise_file[self.noise_type])}')
            self.noisy_labels = noise_file[self.noise_type].reshape(-1)
        else:
            raise Exception('Input Error')

    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        if self.train and self.noise_type is not None:
            target = self.noisy_labels[index]
        return img, target, index

In [11]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, params):
        super().__init__()
        self.seed = params['seed']
        self.batch_size = params['bs']
        self.num_workers = params['num_workers']
        self.mean = params['mean']
        self.std = params['std']
        self.ns = params['num_samples']
        self.rand_res_crop = params['randResCrop']
        self.noise_path = params.get('noise_path', './data/CIFAR-10_human.pt')
        self.noise_type = params.get('noise_type', 'worse_label')  # Default to 'worse_label'

        # Ensure the data directory exists
        os.makedirs(os.path.dirname(self.noise_path), exist_ok=True)

        # Download the CIFAR-10_human.pt file if it doesn't exist
        if not os.path.exists(self.noise_path):
            print(f"Downloading CIFAR-10_human.pt from GitHub...")
            download_file(
                url="https://github.com/UCSC-REAL/cifar-10-100n/raw/main/data/CIFAR-10_human.pt",
                save_path=self.noise_path
            )

        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(size=self.rand_res_crop['size'],
                                         scale=self.rand_res_crop['scale'],
                                         ratio=self.rand_res_crop['ratio']),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

    def prepare_data(self):
        # Download CIFAR-10 dataset
        datasets.CIFAR10(root='./data', train=True, download=True)
        datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        # Load noisy labels
        noise_file = torch.load(self.noise_path)
        clean_label = noise_file['clean_label']
        noisy_label = noise_file[self.noise_type]

        # Split dataset into train and validation sets
        cifar10_full = CIFAR10(root='./data', train=True, transform=self.transform,
                               noise_type=self.noise_type, noise_path=self.noise_path, is_human=True)
        pl.seed_everything(self.seed)
        self.cifar10_train, self.cifar10_val = random_split(cifar10_full,
                                                            [self.ns['train'],
                                                             self.ns['val']])
        self.cifar10_test = CIFAR10(root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.cifar10_train, batch_size=self.batch_size,
                          num_workers=self.num_workers, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.cifar10_val, batch_size=self.batch_size,
                          num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.cifar10_test, batch_size=self.batch_size,
                          shuffle=False)

Training module

In [12]:
class train_model(pl.LightningModule):
    def __init__(self, model=None, loss=None, hparams=hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.model = model
        self.loss_fn = loss
        self.nc = hparams['n_classes']
        self.lr = hparams['lr']
        self.wd = hparams['weight_decay']

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y, _ = batch  # Unpack batch (ignore indices for now)
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Log training loss and accuracy
        # preds = torch.argmax(logits[:, :self.nc], dim=1)
        # acc = (preds == y).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        # self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch  # Unpack batch (ignore indices for now)
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Log validation loss and accuracy
        # preds = torch.argmax(logits[:, :self.nc], dim=1)
        # acc = (preds == y).float().mean()
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        # self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y, _ = batch  # Unpack batch (ignore indices for now)
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Log test loss and accuracy
        preds = torch.argmax(logits[:, :self.nc], dim=1)
        acc = (preds == y).float().mean()
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return {'loss': loss, 'preds': preds, 'y': y}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)

        # Optionally, add a learning rate scheduler
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1.0)
        return [optimizer], [scheduler]

### Models

CNN from paper by [Xia](https://arxiv.org/abs/2106.00445)

In [13]:
def call_bn(bn, x):
    return bn(x)

class CNN(nn.Module):
    def __init__(self, input_channel=3, n_outputs=10, dropout_rate=0.25, top_bn=False):
        self.dropout_rate = dropout_rate
        self.top_bn = top_bn
        super(CNN, self).__init__()
        self.c1=nn.Conv2d(input_channel,128,kernel_size=3,stride=1, padding=1)
        self.c2=nn.Conv2d(128,128,kernel_size=3,stride=1, padding=1)
        self.c3=nn.Conv2d(128,128,kernel_size=3,stride=1, padding=1)
        self.c4=nn.Conv2d(128,256,kernel_size=3,stride=1, padding=1)
        self.c5=nn.Conv2d(256,256,kernel_size=3,stride=1, padding=1)
        self.c6=nn.Conv2d(256,256,kernel_size=3,stride=1, padding=1)
        self.c7=nn.Conv2d(256,512,kernel_size=3,stride=1, padding=0)
        self.c8=nn.Conv2d(512,256,kernel_size=3,stride=1, padding=0)
        self.c9=nn.Conv2d(256,128,kernel_size=3,stride=1, padding=0)
        self.l_c1=nn.Linear(128,n_outputs)
        self.bn1=nn.BatchNorm2d(128)
        self.bn2=nn.BatchNorm2d(128)
        self.bn3=nn.BatchNorm2d(128)
        self.bn4=nn.BatchNorm2d(256)
        self.bn5=nn.BatchNorm2d(256)
        self.bn6=nn.BatchNorm2d(256)
        self.bn7=nn.BatchNorm2d(512)
        self.bn8=nn.BatchNorm2d(256)
        self.bn9=nn.BatchNorm2d(128)

    def forward(self, x,):
        h=x
        h=self.c1(h)
        h=F.leaky_relu(call_bn(self.bn1, h), negative_slope=0.01)
        h=self.c2(h)
        h=F.leaky_relu(call_bn(self.bn2, h), negative_slope=0.01)
        h=self.c3(h)
        h=F.leaky_relu(call_bn(self.bn3, h), negative_slope=0.01)
        h=F.max_pool2d(h, kernel_size=2, stride=2)
        h=F.dropout2d(h, p=self.dropout_rate)

        h=self.c4(h)
        h=F.leaky_relu(call_bn(self.bn4, h), negative_slope=0.01)
        h=self.c5(h)
        h=F.leaky_relu(call_bn(self.bn5, h), negative_slope=0.01)
        h=self.c6(h)
        h=F.leaky_relu(call_bn(self.bn6, h), negative_slope=0.01)
        h=F.max_pool2d(h, kernel_size=2, stride=2)
        h=F.dropout2d(h, p=self.dropout_rate)

        h=self.c7(h)
        h=F.leaky_relu(call_bn(self.bn7, h), negative_slope=0.01)
        h=self.c8(h)
        h=F.leaky_relu(call_bn(self.bn8, h), negative_slope=0.01)
        h=self.c9(h)
        h=F.leaky_relu(call_bn(self.bn9, h), negative_slope=0.01)
        h=F.avg_pool2d(h, kernel_size=h.data.shape[2])

        h = h.view(h.size(0), h.size(1))
        logit=self.l_c1(h)
        if self.top_bn:
            logit=call_bn(self.bn_c1, logit)
        return logit

ResNet50

In [14]:
class ResNet50(nn.Module):
    def __init__(self, n_outputs, freeze=False):
        """
        Args:
            n_outputs (int): Number of output classes.
            freeze (bool): If True, freeze all layers except the head.
        """
        super(ResNet50, self).__init__()
        self.n_outputs = n_outputs
        self.freeze = freeze

        # Load the pre-trained ResNet50 model
        self.resnet50 = models.resnet50(pretrained=True)

        # Modify the final layer to match the number of outputs
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, n_outputs)

        # Freeze all layers except the head if freeze=True
        if self.freeze:
            self._freeze_layers()

    def _freeze_layers(self):
        """
        Freeze all layers except the head.
        """
        # Freeze all parameters in the model
        for param in self.resnet50.parameters():
            param.requires_grad = False

        # Unfreeze the final classification layer (head)
        for param in self.resnet50.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.resnet50(x)

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# # Example usage
# model_frozen = ResNet50(n_outputs=10, freeze=True)
# print(f"Trainable parameters (freeze=True): {count_trainable_parameters(model_frozen)}")

# model_unfrozen = ResNet50(n_outputs=10, freeze=False)
# print(f"Trainable parameters (freeze=False): {count_trainable_parameters(model_unfrozen)}")

ViT

In [15]:
class ViT(nn.Module):
    def __init__(self, n_outputs, freeze=False):
        """
        Args:
            n_outputs (int): Number of output classes.
            freeze (bool): If True, freeze all layers except the head.
        """
        super(ViT, self).__init__()
        self.n_outputs = n_outputs
        self.freeze = freeze

        # Load the pre-trained ViT model
        self.vit = models.vit_b_16(pretrained=True)

        # Modify the final layer to match the number of outputs
        self.vit.heads.head = nn.Linear(self.vit.heads.head.in_features, n_outputs)

        # Freeze all layers except the head if freeze=True
        if self.freeze:
            self._freeze_layers()

    def _freeze_layers(self):
        """
        Freeze all layers except the head.
        """
        # Freeze all parameters in the model
        for param in self.vit.parameters():
            param.requires_grad = False

        # Unfreeze the head (final classification layer)
        for param in self.vit.heads.head.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.vit(x)

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# # Example usage
# model_frozen = ViT(n_outputs=10, freeze=True)
# print(f"Trainable parameters (freeze=True): {count_trainable_parameters(model_frozen)}")

# model_unfrozen = ViT(n_outputs=10, freeze=False)
# print(f"Trainable parameters (freeze=False): {count_trainable_parameters(model_unfrozen)}")

# class ViT(nn.Module):
#     def __init__(self, n_outputs):
#         super(ViT, self).__init__()
#         self.n_outputs = n_outputs
#         # Load the pre-trained ViT model
#         self.vit = models.vit_b_16(pretrained=True)

#         # Modify the final layer to match the number of outputs
#         self.vit.heads.head = nn.Linear(self.vit.heads.head.in_features, n_outputs)

#     def forward(self, x):
#         return self.vit(x)

### Loss functions

Create a loss function class, or use a standart one.

In [16]:
# Cross entropy loss maden from scratch (just in case)
class CELoss(nn.Module):
    def __init__(self, params=hparams):
        super(CELoss, self).__init__()
        self.smoothing = params.get('label_smoothing', 0.0)  # Default smoothing value
        self.num_classes = params.get('n_classes', 10)
        self.inv_smoothing = 1.0 - self.smoothing  # Probability for the correct class

    def forward(self, x, y):
        """
        x: Model output (logits)
            - Shape: (batch_size, num_classes)
        y: Labels
            - Shape: (batch_size,)
        """
        # Apply label smoothing to the one-hot encoded labels
        with torch.no_grad():
            yoh = torch.zeros_like(x)  # Create a one-hot encoded version of y
            yoh.fill_(self.smoothing / (self.num_classes - 1))  # Fill with smoothed values
            yoh.scatter_(1, y.unsqueeze(1), self.inv_smoothing)  # Set correct class to 1 - smoothing

        # Compute the cross-entropy loss between logits and smoothed labels
        log_probs = F.log_softmax(x, dim=1)  # Log probabilities
        loss = -(yoh * log_probs).sum(dim=1).mean()  # Sum over classes and mean over batch

        return loss

In [17]:
class NLoss(nn.Module):
    def __init__(self, params=hparams):
        super(NLoss, self).__init__()
        self.smoothing =   params.get('label_smoothing', 0.0)
        self.num_classes = params.get('n_classes', 10)
        self.inv_smoothing = 1.0 - self.smoothing  # Probability for the correct class

    def forward(self, x, y):
        """
        x: Model output (logits + log variance)
            - x[:, :self.num_classes]: Logits for class probabilities (h)
            - x[:, self.num_classes:]: Logarithmic variance (s)
        y: Labels
        """
        # Split the model output into predictions (h) and log variance (s)
        logits = x[:, :self.num_classes]  # Predictions (h)
        log_var = x[:, self.num_classes:]  # Logarithmic variance (s)

        # Apply label smoothing to the one-hot encoded labels
        with torch.no_grad():
            yoh = torch.zeros_like(logits)
            yoh.fill_(self.smoothing / (self.num_classes - 1))
            yoh.scatter_(1, y.data.unsqueeze(1), self.inv_smoothing)

        # Compute the squared differences between predictions and smoothed labels
        squared_diff = torch.pow(yoh - logits, 2)  # (y_k - h_k)^2

        # Compute the exponential of the negative log variance (e^{-s})
        exp_neg_log_var = torch.exp(-log_var)

        # Compute the first term of the loss: e^{-s} * sum((y_k - h_k)^2)
        term1 = exp_neg_log_var * squared_diff.sum(dim=1)

        # Compute the second term of the loss: N * s
        term2 = self.num_classes * log_var

        # Combine the terms and compute the mean over the batch
        loss = (term1 + term2).mean()

        return loss

In [18]:
class BLoss(nn.Module):
    def __init__(self, params=hparams):
        super(BLoss, self).__init__()
        self.smoothing =   params.get('label_smoothing', 0.0)
        self.num_classes = params.get('n_classes', 10)
        self.inv_smoothing = 1.0 - self.smoothing  # Probability for the correct class


    def forward(self, x, y):
        # Extract certainty and probabilities from the model output
        certainty = torch.sigmoid(x[:, self.num_classes:])  # Certainty values
        logits = x[:, :self.num_classes]  # Logits for class probabilities
        prob = F.softmax(logits, dim=1)  # Softmax probabilities

        # Compute cosine similarity between predictions and labels
        cos = nn.CosineSimilarity(dim=1)

        # Apply label smoothing to the one-hot encoded labels
        with torch.no_grad():
            yoh = torch.zeros_like(logits)
            yoh.fill_(self.smoothing / (self.num_classes - 1))
            yoh.scatter_(1, y.data.unsqueeze(1), self.inv_smoothing)


        # Compute the terms of the loss
        cosyh = cos(yoh, prob)
        delta = yoh * prob  # Element-wise product of one-hot labels and probabilities
        entropy_term = delta * torch.log(delta + 1e-10)  # Entropy term (avoid log(0))

        # Loss terms
        loss0 = -cosyh * torch.log(certainty / self.num_classes + 1e-10)  # First term
        loss1 = -(self.num_classes - 1) * (1 - cosyh) * torch.log((1 - certainty) / self.num_classes + 1e-10)  # Second term

        # Combine the terms and compute the mean over the batch
        loss = (entropy_term.sum(dim=1) + loss0 + loss1).mean()

        return loss

### Models zoo

Architectures and loss functions

In [19]:
def get_arch_and_loss(hparams):
    """
    Returns the architecture and loss function based on the provided hparams.

    Args:
        hparams (dict): Hyperparameters dictionary, including 'ARCHITECTURE' and 'criterion'.

    Returns:
        arch: The model architecture.
        loss: The loss function.
    """
    # Determine the number of outputs based on the loss function
    if hparams['criterion'] in ['B', 'N']:
        n_outputs = hparams['n_classes'] + 1  # Add 1 output neuron for BLoss or NLoss
    else:
        n_outputs = hparams['n_classes']  # Default number of outputs

    # Define the architectures
    architectures = {
        'CNN': CNN(n_outputs=n_outputs),
        'ResNet50': ResNet50(n_outputs=n_outputs, freeze=hparams.get('freeze', False)),
        'ViT': ViT(n_outputs=n_outputs, freeze=hparams.get('freeze', False)),
    }

    # Define the loss functions
    losses = {
        'CE':CELoss(),
        'B': BLoss(),
        'N': NLoss(),
    }

    # Get the architecture and loss based on hparams
    arch = architectures.get(hparams['architecture'])
    loss = losses.get(hparams['criterion'])

    if arch is None:
        raise ValueError(f"Architecture '{hparams['ARCHITECTURE']}' is not supported.")
    if loss is None:
        raise ValueError(f"Loss function '{hparams['criterion']}' is not supported.")

    return arch, loss


### Metrics

In [20]:
def metrics(dataloader,model,hparams=hparams,loss_fn_red=None):
    # Collect images, predictions, and losses
    # images = []
    preds  = []
    labels = []
    losses = []
    confidences = []
    correct= 0
    total  = 0
    for batch in dataloader:
        x, y, _ = batch
        with torch.no_grad():
            logits = model(x)
            # loss = loss_fn_red(h,y)
            pred = torch.argmax(logits[:,:hparams['n_classes']], dim=1)
            confidence =  logits[:,hparams['n_classes']:]
        correct += (pred == y).sum().item()  # Number of correct predictions
        total += y.size(0)  # Total number of samples

        # images.extend(x.cpu())
        preds.extend(pred.cpu().numpy())
        labels.extend(y.cpu().numpy())
        confidences.extend(confidence.cpu().numpy())
        
        # losses.extend(loss.cpu().numpy())
    acc = correct / total
    return preds, labels, acc, confidences

### Visualization


In [21]:
# Plot confusion matrix
def conf_mat(figsize,class_names=None):
    plt.figure(figsize)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Ensembling
This approach is expected to give a robust ensemble model that leverages the diversity introduced by different seeds, potentially improving the overall accuracy on the test set.

## Create Dataset and Data Loaders

Initialization of the dataset, the dataloader, and the training module

In [22]:
data_module = CIFAR10DataModule(hparams)

File downloaded and saved to ./data/CIFAR-10_human.pt


## Train the Ensemble

Loop over different seeds

In [23]:
# List to store predictions from each model
all_predictions = []
all_confidences = []

In [24]:
for seed in SEEDS:
    # Set seed for reproducibility at the VERY BEGINNING
    pl.seed_everything(seed)

    # Reinitialize the model architecture for each seed
    arch, loss_fn = get_arch_and_loss(hparams)

    checkpoint_callback_img = ModelCheckpoint(
        monitor='val_loss',       # Monitor validation loss
        dirpath=CHECKPOINT_PATH,  # Directory to save checkpoints
        filename=f'best_model_{ARCHITECTURE}_{LOSS_FUN}_{seed}_{LS}_{NOISE_TYPE}',  # Checkpoint filename
        save_top_k=1,             # Save only the best model
        mode='min',               # Minimize validation loss
    )

    task = Task.init(project_name="ICML-2025",
                     task_name=f'arch_{ARCHITECTURE}_loss_{LOSS_FUN}_seed_{seed}_LS_{LS}_noise_{NOISE_TYPE}_redo')

    # Initialize the model with the reinitialized architecture
    model = train_model(model=arch, loss=loss_fn)

    # Log hyperparameters to ClearML
    task.connect(model.hparams)

    trainer = Trainer(max_epochs=hparams['num_epochs'],
                      callbacks=[checkpoint_callback_img],
                      accelerator="auto", devices="auto")
    trainer.fit(model, data_module)

    best_model_path = checkpoint_callback_img.best_model_path
    task.update_output_model(model_path=best_model_path, auto_delete_file=False)
    best_model = train_model.load_from_checkpoint(best_model_path,
                                                  model=arch,
                                                  loss=loss_fn)

    # Test set
    test_dataloader = data_module.test_dataloader()
    # Move the model to the correct device
    best_model = best_model.to(device)
    predictions = []
    confidences = []
    with torch.no_grad():
        for batch in test_dataloader:
            x, _, _, = batch  # We only need the input data, not the labels
            logits = best_model(x.to(device))
            preds = torch.argmax(logits[:, :NUM_CLASSES], dim=1)
            predictions.append(preds.cpu().numpy())
            confs = logits[:, NUM_CLASSES:]
            confidences.append(confs.cpu().numpy())
    predictions = np.concatenate(predictions)  # Combine all batch predictions
    confidences = np.concatenate(confidences)
    all_predictions.append(predictions)
    all_confidences.append(confidences)
    task.close()
    del[model, best_model, task, arch, loss_fn]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 208MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

ClearML Task: created new task id=95e71e1c1d6745f3936dffb7a48cc3cb
2025-03-16 14:30:54,821 - clearml.Repository Detection - WARNING - Jupyter Notebook auto-logging failed, could not access: /kaggle/working/__notebook_source__.ipynb
2025-03-16 14:30:54,872 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/51900be064234ad796d9ef1c21d0576f/experiments/95e71e1c1d6745f3936dffb7a48cc3cb/output/log
2025-03-16 14:31:12,627 - clearml.Task - WARNING - Parameters must be of builtin type (General/mean[ndarray], General/std[ndarray])


100%|██████████| 170M/170M [00:02<00:00, 84.2MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/clearml/binding/frameworks/pytorch_bind.py:277: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

██████████████████████████████▉ 100% | 50.82/50.82 MB [00:00<00:00, 66.12MB/s]: 
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:209: UserWarning:

Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:209: UserWarning:

Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.



## Test the models and the ensemble of the models

In [ ]:
task = Task.init(project_name="ICML-2025",
                     task_name=f'arch_{ARCHITECTURE}_loss_{LOSS_FUN}_LS_{LS}_noise_{NOISE_TYPE}_ensemble_redo')

all_predictions = []
all_confidences = []
for seed in SEEDS:

    # Reinitialize the model architecture for each seed
    arch, loss_fn = get_arch_and_loss(hparams)

    # Initialize the model with the reinitialized architecture
    model = train_model(model=arch, loss=loss_fn)

    best_model_path = "/kaggle/working/saved_models/" +f'best_model_{ARCHITECTURE}_{LOSS_FUN}_{seed}_{LS}_{NOISE_TYPE}.ckpt'
    best_model = train_model.load_from_checkpoint(best_model_path,
                                                  model=arch,
                                                  loss=loss_fn)

    # Test set
    test_dataloader = data_module.test_dataloader()
    # Move the model to the correct device
    best_model = best_model.to(device)
    predictions = []
    confidences = []
    with torch.no_grad():
        for batch in test_dataloader:
            x, _, _, = batch  # We only need the input data, not the labels
            logits = best_model(x.to(device))
            preds = torch.argmax(logits[:, :NUM_CLASSES], dim=1)
            predictions.append(preds.cpu().numpy())
            confs = logits[:, NUM_CLASSES:]
            confidences.append(confs.cpu().numpy())
    predictions = np.concatenate(predictions)  # Combine all batch predictions
    confidences = np.concatenate(confidences)
    all_predictions.append(predictions)
    all_confidences.append(confidences)
    del[model, best_model, arch, loss_fn]


# List to store individual model accuracies
individual_accuracies = []

# Compute accuracy for each model
for i, predictions in enumerate(all_predictions):
    # Get predictions for the current model
    model_predictions = predictions  # Shape: (num_samples,)

    # Get true labels (already collected earlier)
    true_labels = np.array(data_module.cifar10_test.targets)

    # Calculate accuracy for the current model
    accuracy = accuracy_score(true_labels, model_predictions)
    individual_accuracies.append(accuracy)
    print(f'Model {i+1} Accuracy: {accuracy:.4f}')

# Convert to numpy array for easier calculations
individual_accuracies = np.array(individual_accuracies)

# Compute mean accuracy
mean_accuracy = np.mean(individual_accuracies)

# Compute standard deviation of accuracy
std_accuracy = np.std(individual_accuracies)

print(f'Mean Accuracy: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy: {std_accuracy:.4f}')

# Stack predictions from all models
all_predictions = np.stack(all_predictions)  # Shape: (num_models, num_samples, num_classes)
all_confidences = np.hstack(all_confidences)

test_labels = np.array(data_module.cifar10_test.targets)

aggregated_confidences = aggregate_confidences(all_predictions, all_confidences, NUM_CLASSES)
ensemble_weighted_predictions = np.argmax(aggregated_confidences, axis = 1)

accuracy_weight = accuracy_score(test_labels, ensemble_weighted_predictions)
print(f'Ensemble Accuracy: {accuracy_weight:.4f}')

# Ensemble predictions (e.g., by averaging)
ensemble_predictions = np.mean(all_predictions, axis=0)  # Shape: (num_samples, num_classes)
final_predictions, _ = mode(all_predictions, axis=0)  # Majority voting
final_predictions = final_predictions.flatten()  # Flatten to 1D array


# test_labels = data_module.test_dataset.labels  # Adjust this based on your dataset

# Calculate accuracy
accuracy = accuracy_score(test_labels, final_predictions)

# Simulated test metrics
test_metrics = {
    "Mean Accuracy (individual)": mean_accuracy,
    "Standard Deviation of Accuracy (individual)": std_accuracy,
    "Ensemble Accuracy": accuracy,
    "Weighted Ensemble Accuracy": accuracy_weight,
}
task.connect(test_metrics)
task.close()

In [36]:
task = Task.init(project_name="ICML-2025",
                     task_name=f'arch_{ARCHITECTURE}_loss_{LOSS_FUN}_LS_{LS}_noise_{NOISE_TYPE}_ensemble_redo')

Jupyter Notebook auto-logging failed, could not access: /kaggle/working/__notebook_source__.ipynb


ClearML Task: overwriting (reusing) task id=a3d7f62893d940538e1e289bc1afb4e9
ClearML results page: https://app.clear.ml/projects/51900be064234ad796d9ef1c21d0576f/experiments/a3d7f62893d940538e1e289bc1afb4e9/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [45]:
all_predictions = []
all_confidences = []
for seed in SEEDS:

    # Reinitialize the model architecture for each seed
    arch, loss_fn = get_arch_and_loss(hparams)

    # Initialize the model with the reinitialized architecture
    model = train_model(model=arch, loss=loss_fn)

    best_model_path = "/kaggle/working/saved_models/" +f'best_model_{ARCHITECTURE}_{LOSS_FUN}_{seed}_{LS}_{NOISE_TYPE}.ckpt'
    best_model = train_model.load_from_checkpoint(best_model_path,
                                                  model=arch,
                                                  loss=loss_fn)

    # Test set
    test_dataloader = data_module.test_dataloader()
    # Move the model to the correct device
    best_model = best_model.to(device)
    predictions = []
    confidences = []
    with torch.no_grad():
        for batch in test_dataloader:
            x, _, _, = batch  # We only need the input data, not the labels
            logits = best_model(x.to(device))
            preds = torch.argmax(logits[:, :NUM_CLASSES], dim=1)
            predictions.append(preds.cpu().numpy())
            confs = logits[:, NUM_CLASSES:]
            confidences.append(confs.cpu().numpy())
    predictions = np.concatenate(predictions)  # Combine all batch predictions
    confidences = np.concatenate(confidences)
    all_predictions.append(predictions)
    all_confidences.append(confidences)
    del[model, best_model, arch, loss_fn]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.

Connecting multiple input m

Individual models

In [46]:

# List to store individual model accuracies
individual_accuracies = []

# Compute accuracy for each model
for i, predictions in enumerate(all_predictions):
    # Get predictions for the current model
    model_predictions = predictions  # Shape: (num_samples,)

    # Get true labels (already collected earlier)
    true_labels = np.array(data_module.cifar10_test.targets)

    # Calculate accuracy for the current model
    accuracy = accuracy_score(true_labels, model_predictions)
    individual_accuracies.append(accuracy)
    print(f'Model {i+1} Accuracy: {accuracy:.4f}')

# Convert to numpy array for easier calculations
individual_accuracies = np.array(individual_accuracies)

# Compute mean accuracy
mean_accuracy = np.mean(individual_accuracies)

# Compute standard deviation of accuracy
std_accuracy = np.std(individual_accuracies)

print(f'Mean Accuracy: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy: {std_accuracy:.4f}')

Model 1 Accuracy: 0.8591
Model 2 Accuracy: 0.8561
Model 3 Accuracy: 0.8582
Model 4 Accuracy: 0.8597
Model 5 Accuracy: 0.8572
Model 6 Accuracy: 0.8550
Model 7 Accuracy: 0.8579
Mean Accuracy: 0.8576
Standard Deviation of Accuracy: 0.0015


Ensemble

In [47]:
# Stack predictions from all models
all_predictions = np.stack(all_predictions)  # Shape: (num_models, num_samples, num_classes)
all_confidences = np.hstack(all_confidences)

In [48]:
all_confidences.shape

(10000, 7)

In [49]:
import numpy as np
def aggregate_confidences(predictions, confidences, n_classes):
    """
    Aggregate confidences per class according to the provided formula.
    
    Arguments:
    predictions -- np.ndarray of shape (n_models, n_samples), predictions from the models
    confidences -- np.ndarray of shape (n_models, n_samples), confidence scores of models
    n_classes -- int, number of classes
    
    Returns:
    aggregated_confidences -- np.ndarray of shape (n_samples, n_classes)
    """
    n_models, n_samples = predictions.shape  # Get the dimensions of the data
    
    # Initialize an array to hold the aggregated confidences for each class
    aggregated_confidences = np.zeros((n_samples, n_classes))
    
    # Loop through each class
    for k in range(n_classes):
        # Compute the indicator matrix: shape (n_models, n_samples)
        indicator = (predictions == k).astype(float)  # Convert True/False to 1/0
        
        # Multiply confidences by the indicator function and sum over models
        # Resulting shape: (n_samples,)
        class_confidence = np.sum(confidences * indicator, axis=0)
        
        # Store the aggregated confidence for class k
        aggregated_confidences[:, k] = class_confidence
    
    return aggregated_confidences

In [50]:
# Get true labels from the CIFAR-10 data set
test_labels = np.array(data_module.cifar10_test.targets)

In [51]:
aggregated_confidences = aggregate_confidences(all_predictions, all_confidences.T, NUM_CLASSES)
ensemble_weighted_predictions = np.argmax(aggregated_confidences, axis = 1)

accuracy_weight = accuracy_score(test_labels, ensemble_weighted_predictions)
print(f'Ensemble Accuracy: {accuracy_weight:.4f}')

# Compute confusion matrix
cm = confusion_matrix(test_labels, ensemble_weighted_predictions)
cm

Ensemble Accuracy: 0.8977


array([[923,   4,  14,   7,   5,   3,   3,   5,  29,   7],
       [  3, 960,   0,   2,   1,   2,   2,   1,   7,  22],
       [ 36,   1, 830,  27,  33,  20,  29,  14,   6,   4],
       [ 14,   3,  34, 775,  26,  84,  37,  15,   4,   8],
       [  4,   1,  18,  19, 909,  12,  15,  17,   5,   0],
       [  8,   0,  24,  85,  23, 833,   8,  18,   0,   1],
       [  6,   0,  12,  28,   4,   6, 937,   3,   3,   1],
       [  5,   0,   6,  16,  25,  14,   1, 928,   2,   3],
       [ 27,  10,   4,   1,   1,   3,   2,   0, 945,   7],
       [  7,  29,   1,   3,   2,   2,   4,   5,  10, 937]])

In [52]:
# Ensemble predictions (e.g., by averaging)
ensemble_predictions = np.mean(all_predictions, axis=0)  # Shape: (num_samples, num_classes)
final_predictions, _ = mode(all_predictions, axis=0)  # Majority voting
final_predictions = final_predictions.flatten()  # Flatten to 1D array


# test_labels = data_module.test_dataset.labels  # Adjust this based on your dataset

# Calculate accuracy
accuracy = accuracy_score(test_labels, final_predictions)
print(f'Ensemble Accuracy: {accuracy:.4f}')

# Compute confusion matrix
cm = confusion_matrix(test_labels, final_predictions)

Ensemble Accuracy: 0.8975


In [53]:
# Simulated test metrics
test_metrics = {
    "Mean Accuracy (individual)": mean_accuracy,
    "Standard Deviation of Accuracy (individual)": std_accuracy,
    "Ensemble Accuracy": accuracy,
    "Weighted Ensemble Accuracy": accuracy_weight,
}

In [54]:
task.connect(test_metrics)
test_metrics

{'Mean Accuracy (individual)': 0.8576,
 'Standard Deviation of Accuracy (individual)': 0.001525029273723724,
 'Ensemble Accuracy': 0.8975,
 'Weighted Ensemble Accuracy': 0.8977}

In [55]:
task.close()

Code for N-loss case where we need to aplly sigmoid

In [ ]:
task = Task.init(project_name="ICML-2025",
                     task_name=f'arch_{ARCHITECTURE}_loss_{LOSS_FUN}_LS_{LS}_noise_{NOISE_TYPE}_ensemble_redo')

all_predictions = []
all_confidences = []
for seed in SEEDS:

    # Reinitialize the model architecture for each seed
    arch, loss_fn = get_arch_and_loss(hparams)

    # Initialize the model with the reinitialized architecture
    model = train_model(model=arch, loss=loss_fn)

    best_model_path = "/kaggle/working/saved_models/" +f'best_model_{ARCHITECTURE}_{LOSS_FUN}_{seed}_{LS}_{NOISE_TYPE}.ckpt'
    best_model = train_model.load_from_checkpoint(best_model_path,
                                                  model=arch,
                                                  loss=loss_fn)

    # Test set
    test_dataloader = data_module.test_dataloader()
    # Move the model to the correct device
    best_model = best_model.to(device)
    predictions = []
    confidences = []
    with torch.no_grad():
        for batch in test_dataloader:
            x, _, _, = batch  # We only need the input data, not the labels
            logits = best_model(x.to(device))
            preds = torch.argmax(logits[:, :NUM_CLASSES], dim=1)
            predictions.append(preds.cpu().numpy())
            confs = logits[:, NUM_CLASSES:]
            confidences.append(confs.cpu().numpy())
    predictions = np.concatenate(predictions)  # Combine all batch predictions
    confidences = np.concatenate(confidences)
    all_predictions.append(predictions)
    all_confidences.append(confidences)
    del[model, best_model, arch, loss_fn]


# List to store individual model accuracies
individual_accuracies = []

# Compute accuracy for each model
for i, predictions in enumerate(all_predictions):
    # Get predictions for the current model
    model_predictions = predictions  # Shape: (num_samples,)

    # Get true labels (already collected earlier)
    true_labels = np.array(data_module.cifar10_test.targets)

    # Calculate accuracy for the current model
    accuracy = accuracy_score(true_labels, model_predictions)
    individual_accuracies.append(accuracy)
    print(f'Model {i+1} Accuracy: {accuracy:.4f}')

# Convert to numpy array for easier calculations
individual_accuracies = np.array(individual_accuracies)

# Compute mean accuracy
mean_accuracy = np.mean(individual_accuracies)

# Compute standard deviation of accuracy
std_accuracy = np.std(individual_accuracies)

print(f'Mean Accuracy: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy: {std_accuracy:.4f}')

# Stack predictions from all models
all_predictions = np.stack(all_predictions)  # Shape: (num_models, num_samples, num_classes)
all_confidences = np.hstack(all_confidences)

def sigmoid(z):
    return 1/(1 + np.exp(-z))

all_confidences = 1- sigmoid(all_confidences.T)

test_labels = np.array(data_module.cifar10_test.targets)

aggregated_confidences = aggregate_confidences(all_predictions, all_confidences, NUM_CLASSES)
ensemble_weighted_predictions = np.argmax(aggregated_confidences, axis = 1)

accuracy_weight = accuracy_score(test_labels, ensemble_weighted_predictions)
print(f'Ensemble Accuracy: {accuracy_weight:.4f}')

# Ensemble predictions (e.g., by averaging)
ensemble_predictions = np.mean(all_predictions, axis=0)  # Shape: (num_samples, num_classes)
final_predictions, _ = mode(all_predictions, axis=0)  # Majority voting
final_predictions = final_predictions.flatten()  # Flatten to 1D array


# test_labels = data_module.test_dataset.labels  # Adjust this based on your dataset

# Calculate accuracy
accuracy = accuracy_score(test_labels, final_predictions)

# Simulated test metrics
test_metrics = {
    "Mean Accuracy (individual)": mean_accuracy,
    "Standard Deviation of Accuracy (individual)": std_accuracy,
    "Ensemble Accuracy": accuracy,
    "Weighted Ensemble Accuracy": accuracy_weight,
}
task.connect(test_metrics)
task.close()
